# NVIDIA NIM ಮತ್ತು LlamaIndex ಸಂಯೋಜನೆಯೊಂದಿಗೆ Azure AI Search

In this notebook, we'll demonstrate how to leverage NVIDIA's AI models and LlamaIndex to create a powerful Retrieval-Augmented Generation (RAG) pipeline. We'll use NVIDIA's LLMs and embeddings, integrate them with Azure AI Search as the vector store, and perform RAG to enhance search quality and efficiency.

## ಪ್ರಯೋಜನಗಳು
- **ವಿಸ್ತರಣೀಯತೆ**: NVIDIA ಅತಿ ದೊಡ್ಡ ಭಾಷಾ ಮಾದರಿಗಳನ್ನೂ ಮತ್ತು Azure AI Search ಅನ್ನು ವಿಶ್ವವ್ಯಾಪಿಯಾಗಿ ಮತ್ತು ಪರಿಣಾಮಕಾರಿಯಾಗಿ ಪಡೆಯಲು ಬಳಸಿಕೊಳ್ಳಿ.
- **ఖರ್ಚು ಪರಿಣಾಮಕಾರಿತ್ವ**: ಪರಿಣಾಮಕಾರಿ ವೆಕ್ಟರ್ ಸಂಗ್ರಹಣ ಮತ್ತು ಸಂಶ್ಲೇಷಿತ ಹುಡುಕಾಟ ತಂತ್ರಗಳೊಂದಿಗೆ ಹುಡುಕಾಟ ಮತ್ತು ಪ್ರಾಪ್ತಿ optimizಮಾಡಿ.
- **ಉತ್ತಮ ಕಾರ್ಯಕ್ಷಮತೆ**: ದ್ರುತ ಮತ್ತು ಹೆಚ್ಚು ನಿಖರ ಉತ್ತರಗಳಿಗಾಗಿ ಶಕ್ತಿಶಾಲಿ LLMಗಳನ್ನು ವೆಕ್ಟರೈಸ್ ಮಾಡಿದ ಹುಡುಕಾಟದೊಂದಿಗೆ ಸಂಯೋಜಿಸಿ.
- **ಗುಣಮಟ್ಟ**: ಸಂಬಂಧಿತ ಪಡೆದುಕೊಂಡ ಡಾಕ್ಯುಮೆಂಟ್‌ಗಳೊಂದಿಗೆ LLM ಉತ್ತರಗಳನ್ನು ಅಡಕಮಾಡುವ ಮೂಲಕ ಹೆಚ್ಚಿನ ಹುಡುಕಾಟ ಗುಣಮಟ್ಟವನ್ನು ಕಾಪಾಡಿಕೊಳ್ಳಿ.

## ಅಗತ್ಯಶರತ್ತುಗಳು
- 🐍 Python 3.9 ಅಥವಾ ಹೆಚ್ಚಿನ
- 🔗 [Azure AI Search ಸೇವೆ](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API ಕೀ — NVIDIA NIM ಮೈಕ್ರೋಸರ್ವೀಸ್‌ಗಳ ಮೂಲಕ NVIDIA 的 LLMಗಳು ಮತ್ತು Embeddings ಗೆ ಪ್ರವೇಶಕ್ಕೆ

## ಒಳಗೊಂಡ ವೈಶಿಷ್ಟ್ಯಗಳು
- ✅ NVIDIA LLM ಸಂಯೋಜನೆ (ನಾವು [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe) ಅನ್ನು ಬಳಸುತ್ತೇವೆ)
- ✅ NVIDIA ಎಂಬೆಡ್ಡಿಂಗ್ಸ್ (ನಾವು [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5) ಅನ್ನು ಬಳಸುತ್ತೇವೆ)
- ✅ Azure AI Search ಸುಧಾರಿತ ಪಡೆಯುವಿಕೆ ಮೋಡ್‌ಗಳು
- ✅ LlamaIndex ಬಳಸಿ ಡಾಕ್ಯುಮೆಂಟ್ ಸೂಚ್ಯಂಕೀಕರಣ
- ✅ NVIDIA LLMಗಳೊಂದಿಗೆ Azure AI Search ಮತ್ತು LlamaIndex ಬಳಸಿ RAG

ಆರಂಭಿಸೋಣ!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## ಸ್ಥಾಪನೆ ಮತ್ತು ಆವಶ್ಯಕತೆಗಳು
Python ಆವೃತ್ತಿ >3.10 ಬಳಸಿಕೊಂಡು Python ಪರಿಸರವನ್ನು ರಚಿಸಿ.

## ಪ್ರಾರಂಭಿಸೋಣ!


ಆರಂಭಿಸಲು, NVIDIA AI Foundation ಮಾದರಿಗಳನ್ನು ಬಳಸಲು ನಿಮಗೆ `NVIDIA_API_KEY` ಬೇಕಾಗುತ್ತದೆ:
1) [NVIDIA](https://build.nvidia.com/explore/discover) ನಲ್ಲಿ ಉಚಿತ ಖಾತೆ ರಚಿಸಿ.
2) ನಿಮ್ಮ ಆಯ್ಕೆಯ ಮಾದರಿಯನ್ನು ಕ್ಲಿಕ್ ಮಾಡಿ.
3) Input ವಿಭಾಗದಡಿ, Python ಟ್ಯಾಬ್ ಆಯ್ಕೆ ಮಾಡಿ, ಮತ್ತು **Get API Key** ಅನ್ನು ಕ್ಲಿಕ್ ಮಾಡಿ ಮತ್ತು ನಂತರ **Generate Key** ಅನ್ನು ಕ್ಲಿಕ್ ಮಾಡಿ.
4) ರಚಿಸಿದ ಕೀ ಅನ್ನು ನಕಲಿಸಿ ಮತ್ತು NVIDIA_API_KEY ಎಂದು ಉಳಿಸಿ. ಅಲ್ಲಿ, ನಿಮಗೆ ಎಂಡ್‌ಪಾಯಿಂಟ್‌ಗಳಿಗೆ ಪ್ರವೇಶ ಸಿಗಬೇಕು.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG ಉದಾಹರಣೆ — LLM ಮತ್ತು ಎಂಬೆಡ್ಡಿಂಗ್ ಬಳಸಿ
### 1) LLM ಅನ್ನು ಪ್ರಾರಂಭಿಸಿ
`llama-index-llms-nvidia`, NVIDIA ಯ LLM ಕನೆಕ್ಟರ್ ಎಂದು ಕೂಡ ಪರಿಚಿತವಾಗಿದೆ, ಇದು NVIDIA API ಕ್ಯಾಟಲಾಗ್‌ನಲ್ಲಿ ಲಭ್ಯವಿರುವ ಹೊಂದಾಣಿಕೆಯ ಮಾದರಿಗಳಿಗೆ ಸಂಪರ್ಕ ಸ್ಥಾಪಿಸಿ ಅವುಗಳಿಂದ ರಚನೆ ಮಾಡಲು ನಿಮಗೆ ಅನುಮತಿಸುತ್ತದೆ. ಚಾಟ್ ಪೂರ್ಣಗೊಳಿಸುವ ಮಾದರಿಗಳ ಪಟ್ಟಿ ಇಲ್ಲಿದೆ ನೋಡಿ: https://build.nvidia.com/search?term=Text-to-Text

ಇಲ್ಲಿ ನಾವು **mixtral-8x7b-instruct-v0.1** ಅನ್ನು ಬಳಸುತ್ತೇವೆ


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) ಎಂಬೆಡ್ಡಿಂಗ್ ಅನ್ನು ಪ್ರಾರಂಭಿಸಿ

`llama-index-embeddings-nvidia`, ಎಂದಾಗಲೂ NVIDIA ಯ ಎಂಬೆಡ್ಡಿಂಗ್ಸ್ ಕನೆಕ್ಟರ್ ಎಂದು ಪರಿಚಯಿಸಲಾಗುತ್ತದೆ, ಇದು ನಿಮಗೆ NVIDIA API ಕ್ಯಾಟಾಲಾಗ್‌ನಲ್ಲಿ ಲಭ್ಯವಿರುವ ಹೊಂದಾಣಿಕೆಯ ಮಾದರಿಗಳೊಂದಿಗೆ ಸಂಪರ್ಕ ಸಾಧಿಸಿ ಅವುಗಳಿಂದ ರಚಿಸಲು ಅನುಮತಿಸುತ್ತದೆ. ನಾವು `nvidia/nv-embedqa-e5-v5` ಅನ್ನು ಎಂಬೆಡ್ಡಿಂಗ್ ಮಾದರಿಯಾಗಿ ಆಯ್ಕೆಮಾಡಿದ್ದೇವೆ. ಟೆಕ್ಸ್ಟ್ ಎಂಬೆಡ್ಡಿಂಗ್ ಮಾದರಿಗಳ ಪಟ್ಟಿ ನೋಡಲು ಇಲ್ಲಿ ನೋಡಿ: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Azure AI Search ವೆಕ್ಟರ್ ಸ್ಟೋರ್ ರಚಿಸಿ


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

### 4) ದಾಖಲೆಗಳನ್ನು ಲೋಡ್ ಮಾಡಿ, ವಿಭಾಗಿಸಿ ಮತ್ತು ಅಪ್‌ಲೋಡ್ ಮಾಡಿ


In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) ನಿಮ್ಮ ಡೇಟಾದ ಮೇಲೆ ಪ್ರಶ್ನೆಗಳನ್ನು ಕೇಳಲು ಒಂದು ಕ್ವೇರಿ ಎಂಜಿನ್ ರಚಿಸಿ

ಇದು Azure AI Search ನಲ್ಲಿ ಶುದ್ಧ ವೆಕ್ಟರ್ ಶೋಧನೆಯನ್ನು ಬಳಸಿ, ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ನಮ್ಮ LLM (Phi-3.5-MOE) ಗೆ ನೆಲಭೂತಗೊಳಿಸುವ ಒಂದು ಕ್ವೇರಿಯಾಗಿದೆ


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

ಇಲ್ಲಿದೆ Azure AI Search ನಲ್ಲಿ ಹೈಬ್ರಿಡ್ ಶೋಧನೆಯನ್ನು ಬಳಸಿರುವ ಒಂದು ಪ್ರಶ್ನೆ.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### ವೆಕ್ಟರ್ ಶೋಧ ವಿಶ್ಲೇಷಣೆ
LLM ಪ್ರತಿಕ್ರಿಯೆ ಮೂಲ ಪಠ್ಯದಲ್ಲಿ ಉಲ್ಲೇಖಿಸಲಾದ ರಷ್ಯಾದ ಶೇರು ಮಾರುಕಟ್ಟೆಗೆ ಸಂಬಂಧಿಸಿದ ಪ್ರಮುಖ ಆರ್ಥಿಕ ಪರಿಣಾಮಗಳನ್ನು ಸರಿಯಾಗಿ ಹಿಡಿದಿದೆ. ವಿಶೇಷವಾಗಿ, ಅದು ರಷ್ಯಾದ ಶೇರು ಮಾರುಕಟ್ಟೆ ಮಹತ್ತರ ಕುಸಿತ ಅನುಭವಿಸಿ ಅದರ ಮೌಲ್ಯದ 40% ಕಳೆದುಕೊಂಡಿದ್ದು, جاري ಇರುವ ಪರಿಸ್ಥಿತಿಯ ಕಾರಣದಿಂದ ವ್ಯಾಪಾರ ನಿಲ್ಲಿಸಲಾಯಿತು ಎಂದು ಹೇಳುತ್ತದೆ. ಈ ಪ್ರತಿಕ್ರಿಯೆ ಮೂಲದಲ್ಲಿನ ಮಾಹಿತಿಗೆ ಚೆನ್ನಾಗಿ ಹೊಂದಿಗೊಳ್ಳುತ್ತಿದ್ದು, LLM ಶೇರು ಮಾರುಕಟ್ಟೆ ತೊಂದರೆಯ ಮೇಲೆ ರಷ್ಯಾದ ಕ್ರಮಗಳು ಮತ್ತು ವಿಧಿಸಲಾದ ನಿಷೇಧಗಳ ಪರಿಣಾಮವಾಗಿ ಸಂಬಂಧಿತ ವಿವರಗಳನ್ನು ಸರಿಯಾಗಿ ಗುರುತಿಸಿ ಸಂಕ್ಷಿಪ್ತಗೊಳಿಸಿದ್ದನ್ನು ಸೂಚಿಸುತ್ತದೆ.

#### ಮೂಲ ನೋಡ್‌ಗಳ ಟೀಕೆ
ಮೂಲ ನೋಡ್‌ಗಳು ಆಂತರರಾಷ್ಟ್ರೀಯ ನಿಷೇಧಗಳ ಪರಿಣಾಮವಾಗಿ ರಷ್ಯಾ ಎದುರಿಸಿದ ಆರ್ಥಿಕ ಪರಿಣಾಮಗಳ ವಿವರವಾದ ಚಿತ್ರಣವನ್ನು ಒದಗಿಸುತ್ತವೆ. ಪಠ್ಯವು ರಷ್ಯಾದ ಶೇರು ಮಾರುಕಟ್ಟೆ ಅದರ ಮೌಲ್ಯದ 40% ಕಳೆದುಕೊಂಡ್ದನ್ನು ಮತ್ತು ವ್ಯಾಪಾರ ಸ್ಥಗಿತಗೊಂಡುದಾಗಿ ಬೆಳಗಿಸುತ್ತದೆ. ಜೊತೆಗೆ, ಅದು ರೂಬಲ್‌ನ ಮೌಲ್ಯ ಕುಸಿತ ಮತ್ತು ರಷ್ಯಾದ ಆರ್ಥಿಕತೆಯ ವ್ಯಾಪಕ ಒಂಟಿತನದಂತಹ ಇತರ ಆರ್ಥಿಕ ಪರಿಣಾಮಗಳನ್ನೂ ಉಲ್ಲೇಖಿಸುತ್ತದೆ. LLM ಪ್ರತಿಕ್ರಿಯೆ ಈ ನೋಡ್‌ಗಳ ಪ್ರಮುಖ ಅಂಶಗಳನ್ನು ಪರಿಣಾಮಕಾರಿಯಾಗಿ ಸಂಕ್ಷೇಪಿಸಿ, ಪ್ರಶ್ನೆಯಲ್ಲಿ ಕೇಳಿದ್ದಂತೆ ಶೇರು ಮಾರುಕಟ್ಟೆ上的 ಪರಿಣಾಮದ ಮೇಲೆ ಗಮನ ಕೇಂದ್ರಗೊಳಿಸಿದೆ.


ಈಗ, ಹೈಬ್ರಿಡ್ ಶೋಧನವು ಉತ್ತಮ ಆಧಾರದ ಮೇಲೆ ಉತ್ತರ ನೀಡದಿರುವ ಒಂದು ಪ್ರಶ್ನೆಯನ್ನು ನೋಡೋಣ:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### ಹೈಬ್ರಿಡ್ ಹುಡುಕಾಟ: LLM ಪ್ರತಿಕ್ರಿಯೆ ವಿಶ್ಲೇಷಣೆ
ಹೈಬ್ರಿಡ್ ಹುಡುಕಾಟ ಉದಾಹರಣೆಯಲ್ಲಿನ LLM ಪ್ರತಿಕ್ರಿಯೆಯು ಒದಗಿಸಲಾದ ಸಂದರ್ಭವು ರಷ್ಯಾದ ಯುಕ್ರೇನ್ ಮೇಲೆ ನಡೆದ ಆಕ್ರಮಣದ ನಿಖರ ದಿನಾಂಕವನ್ನು ಸೂಚಿಸುವುದಿಲ್ಲ ಎಂಬುದನ್ನು ಸೂಚಿಸುತ್ತದೆ. ಈ ಪ್ರತಿಕ್ರಿಯೆ LLM ಮೂಲ ದಸ್ತಾವೇಜುಗಳಲ್ಲಿ ಲಭ್ಯವಿರುವ ಮಾಹಿತಿಯನ್ನು ಬಳಸಿಕೊಂಡಿದೆ ಎಂದು示ಿಸುತ್ತದೆ, ಆದರೆ ಪಠ್ಯದಲ್ಲಿ ನಿಖರ ವಿವರಗಳಿಲ್ಲದಿರುವುದನ್ನು ಸ್ವೀಕರಿಸುತ್ತದೆ.

ಈ ಪ್ರತಿಕ್ರಿಯೆ ಸರಿ: ಅದು ಸಂದರ್ಭವು ರಷ್ಯಾದ ಆಕ್ರಮಣಕ್ಕೆ ಸಂಬಂಧಿಸಿದ ಘಟನೆಗಳನ್ನು ಉಲ್ಲೇಖಿಸುತ್ತಿದೆ ಆದರೆ ನಿಖರ ಆಕ್ರಮಣದ ದಿನಾಂಕವನ್ನು ನಿರ್ದಿಷ್ಟಪಡಿಸುವುದಿಲ್ಲ. ಇದು ಒದಗಿಸಿದ ಮಾಹಿತಿಯನ್ನು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವ LLM ಯ ಸಾಮರ್ಥ್ಯವನ್ನು ಮತ್ತು ವಿಷಯದ ಖಾಲಿಗಳನ್ನು ಗುರುತಿಸುವ ಕೌಶಲ್ಯವನ್ನು ಪ್ರದರ್ಶಿಸುತ್ತದೆ. ಮಾಹಿತಿ ಅಪೂರ್ಣವಾಗಿರುವಾಗ LLM ಬಳಕೆದಾರರನ್ನು ನಿಖರ ದಿನಾಂಕಕ್ಕಾಗಿ ಹೊರಗಿನ ಮೂಲಗಳು ಅಥವಾ ಐತಿಹಾಸಿಕ ದಾಖಲೆಗಳನ್ನು ಪರಿಶೀಲಿಸುವಂತೆ ಪ್ರೇರೇಪಿಸುವ ಮೂಲಕ ಎಚ್ಚರಿಕೆ ಪ್ರದರ್ಶಿಸುತ್ತದೆ.

### ಮೂಲ ನೋಡ್‌ಗಳ ವಿಶ್ಲೇಷಣೆ
ಹೈಬ್ರಿಡ್ ಹುಡುಕಾಟ ಉದಾಹರಣೆಯ ಮೂಲ ನೋಡ್‌ಗಳಲ್ಲಿ ಯು.ಎಸ್. ರ ಉತ್ತರವನ್ನು ಚರ್ಚಿಸುವ ಭಾಷಣದ ಕಥನಗಳ ಕೆಲವೊಂದು ಭಾಗಗಳಿವೆ, ಅವು ರಷ್ಯಾದ ಯುಕ್ರೇನ್‌ ಮೇಲೆ ನಡೆದ ಕಾರ್ಯಗಳಿಗೆ ಪ್ರತಿಕ್ರಿಯೆಯಾಗಾಗಿ ಯು.ಎಸ್. ಮತ್ತು ಅದರ ಮೈತ್ರಿಗಳು ಕೈಗೊಂಡಾದ ಕ್ರಮಗಳ ವ್ಯಾಪಕ ಜಿಯೋಪಾಲಿಟಿಕಲ್ ಪರಿಣಾಮವನ್ನು ಮತ್ತು ಕ್ರಮಗಳನ್ನು ಒತ್ತಿಹೇಳುತ್ತವೆ, ಆದರೆ ಅವು ನಿಖರ ಆಕ್ರಮಣದ ದಿನಾಂಕವನ್ನು ಉಲ್ಲೇಖಿಸುವುದಿಲ್ಲ. ಇದು ಸಂದರ್ಭದಲ್ಲಿ ನಿಖರ ದಿನಾಂಕದ ಮಾಹಿತಿ ಬಂದ್ರೆ ಇಲ್ಲವೆಂದು ಸರಿಯಾಗಿ ಗುರುತಿಸಿದ LLM ಪ್ರತಿಕ್ರಿಯೆಯೊಂದಿಗೆ ಹೊಂದಿಕೆಯಾಗುತ್ತದೆ.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### ಹೈಬ್ರಿಡ್ w/ರೀ-ರ್ಯಾಂಕಿಂಗ್: LLM ಪ್ರತಿಕ್ರಿಯೆ ವಿಶ್ಲೇಷಣೆ
ಹೈಬ್ರಿಡ್ w/ರೀ-ರ್ಯಾಂಕಿಂಗ್ ಉದಾಹರಣೆಯಲ್ಲಿ, LLM ಪ್ರತಿಕ್ರಿಯೆ ಭಾಷಣದCompared to earlier? ಹೆಚ್ಚಿನ ಪ್ರಸಂಗಾತ್ಮಕ ಮಾಹಿತಿಯನ್ನು ನೀಡುತ್ತದೆ: ಅದು ಭಾಷಣ ನೀಡಿದಾಗಿನಿಂದ ಆರು ದಿನಗಳ ಮೊದಲು ಆ ಘಟನೆ ಸಂಭವಿಸಿತೆಂದು ಸೂಚಿಸುತ್ತದೆ. ಇದು LLM ಭಾಷಣದ ಸಮಯದ ಆಧಾರದಲ್ಲಿ ಆಕ್ರಮಣದ ದಿನಾಂಕವನ್ನು ಊಹಿಸಲು ಸಾಧ್ಯವೆಂದು ಸೂಚಿಸುತ್ತದೆ, ಆದರೂ ನಿಖರತೆಗೆ ಭಾಷಣದ ನಿಖರ ದಿನಾಂಕವನ್ನು ತಿಳಿದುಕೊಳ್ಳಬೇಕಾಗುತ್ತದೆ.

ಈ ಪ್ರತಿಕ್ರಿಯೆ ಪ್ರಸಂಗ ಸೂಚನೆಗಳನ್ನು ಬಳಸಿಕೊಂಡು ಹೆಚ್ಚು ಮಾಹಿತಿಪೂರ್ಣ ಉತ್ತರ ನೀಡಲು ಸುಧಾರಿತ ಸಾಮರ್ಥ್ಯವನ್ನು ಪ್ರದರ್ಶಿಸುತ್ತದೆ. ಇದು ರೀ-ರ್ಯಾಂಕಿಂಗ್‌ನ ಲಾಭವನ್ನು ಹೀಪಪ್ರಕಾಶಿಸುತ್ತದೆ, ಅಲ್ಲಿ LLM ಹೆಚ್ಚಿನ ಸಂಬಂಧಿತ ಮಾಹಿತಿಯನ್ನು ಪ್ರವೇಶಿಸಿ ಆದ್ಯತೆ ನೀಡುವ ಮೂಲಕ ಬಯಸಿದ ವಿವರದ (ಉದಾಹರಣೆಗೆ, ಆಕ್ರಮಣದ ದಿನಾಂಕ) ಸಮೀಪದ ಅಂದಾಜನ್ನು ನೀಡಲು ಸಾಧ್ಯವಾಗುತ್ತದೆ.

### ಮೂಲ ನೋಡ್‌ಗಳ ವಿಶ್ಲೇಷಣೆ
ಈ ಉದಾಹರಣೆಯಲ್ಲಿನ ಮೂಲ ನೋಡ್‌ಗಳು ರಷ್ಯಾದ ಆಕ್ರಮಣದ ಸಮಯಕ್ಕೆ ಸಂಬಂಧಿಸಿದ ಉಲ್ಲೇಖಗಳನ್ನು ಒಳಗೊಂಡಿವೆ, ವಿಶೇಷವಾಗಿ ಅದು ಭಾಷಣಕ್ಕಿಂತ ಆರು ದಿನಗಳ ಹಿಂದೆ ಸಂಭವಿಸಿತೆಂದು ಹೇಳುವುದನ್ನು. ನಿಖರ ದಿನಾಂಕ ಇನ್ನೂ ಸ್ಪಷ್ಟವಾಗಿ ತಿಳಿಸದಿದ್ದರೂ, ನೋಡ್‌ಗಳು ಕಾಲಾನುಸಾರ ಪ್ರಸಂಗಾತ್ಮಕ ಮಾಹಿತಿ ಒದಗಿಸುತ್ತವೆ, ಇದು LLMಗೆ ಹೆಚ್ಚು ಸೂಕ್ಷ್ಮ ಮತ್ತು ವಿವರಾತ್ಮಕ ಪ್ರತಿಕ್ರಿಯೆ ನೀಡಲು ಸಹಾಯ ಮಾಡುತ್ತದೆ. ಈ ವಿವರವನ್ನು ಸೇರಿಸುವುದು ರೀ-ರ್ಯಾಂಕಿಂಗ್ ಹೇಗೆ LLMನ ಸಾಮರ್ಥ್ಯವನ್ನು ಹೆಚ್ಚಿಸಿ ನೀಡಲಾದ ಪ್ರಸಂಗದಿಂದ ಮಾಹಿತಿ ತೆಗೆದು ಊಹಿಸಲು ಸಹಾಯ ಮಾಡುತ್ತದೆ ಎಂಬುದನ್ನು ತೋರಿಸುತ್ತದೆ, ಪರಿಣಾಮವಾಗಿ ಹೆಚ್ಚು ನಿಖರ ಮತ್ತು ಮಾಹಿತಿಪೂರ್ಣ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ಹೊಂದಿದೆ.


**ಗಮನಿಸಿ:**
ಈ ನೋಟ್‌ಬುಕ್‌ನಲ್ಲಿ, ನಾವು NVIDIA NIM ಮೈಕ್ರೋಸರ್ವಿಸ್ಗಳನ್ನು NVIDIA API Catalog ನಿಂದ ಬಳಸಿದ್ದೇವೆ.
ಮೇಲಿನ API ಗಳು, `NVIDIA (llms)`, `NVIDIAEmbedding`, ಮತ್ತು [Azure AI Search ಅರ್ಥಾತ್ಮಕ ಹೈಬ್ರಿಡ್ ಹುಡುಕಾಟ (ಒಳಗಿರಿಸಲಾಗಿರುವ ಮರುರ‍್ಯಾಂಕಿಂಗ್)](https://learn.microsoft.com/azure/search/semantic-search-overview). ಗಮನಿಸಿ, ಮೇಲಿನ API ಗಳು ಸ್ವಯಂ-ಹೋಸ್ಟ್ ಮಾಡಿದ ಮೈಕ್ರೋಸರ್ವಿಸ್ಗಳನ್ನೂ ಬೆಂಬಲಿಸಬಹುದು. 

**ಉದಾಹರಣೆ:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
ಜವಾಬ್ದಾರಿ ನಿರಾಕರಣೆ:
ಈ ದಸ್ತಾವೇಜನ್ನು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯನ್ನು ಕಾಪಾಡಲು ಪ್ರಯತ್ನಿಸಿದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅನಿಖರತೆಗಳಿರಬಹುದಾಗಿದೆ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿನ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಉಲ್ಲೇಖವಾಗಿ ಪರಿಗಣಿಸಬೇಕು. ಗಂಭೀರ ಮಾಹಿತಿಗಾಗಿ ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದವನ್ನು ಬಳಸಿ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಗ್ರಹಿಕೆಗಳು ಅಥವಾ ತಪ್ಪು ವ್ಯಾಖ್ಯಾನಗಳಿಗೆ ನಾವು ಜವಾಬ್ದಾರರಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
